In [66]:
import utils
import session
import sys
import imp
import time
# imp.reload(session)
imp.reload(utils)
import search

In [26]:
driver = utils.start_driver('chrome')

In [27]:
session.login(driver, 'input/credential.json')

> loading json input/credential.json
> openning URL: https://app.pluralsight.com/id?redirectTo=%2F


In [45]:
def scroll_to_end(driver):
    utils.open_url(driver, 'https://app.pluralsight.com/library/search')
    
    # switch to Course tab
    for elem in driver.find_elements_by_xpath('//li[@class="tab-list__item"]'):
        if elem.text == 'Courses':
            elem.click()
    
    # define target elem
    course_section = driver.find_element_by_xpath('//div[@aria-selected="true"]')
    
    nload = 1
    nload_max = 3
    while True:
        courses = course_section.find_elements_by_xpath('.//li[@class="courses-list__item"]')
        ncourses = len(courses)
        print('load={}, ncourses={}'.format(nload, ncourses))
        
        if nload >= nload_max:
            break
            
        nload += 1
        buttons = course_section.find_elements_by_xpath('.//a[@class="button button--outlined"]')
        if len(buttons) == 0:
            break
        if len(buttons):
            buttons[0].click()
            
        time.sleep(3)
        
    # save html
    utils.save_html(driver, 'courses/all.html')


In [46]:
scroll_to_end(driver)

load=1, ncourses=5451


> writing html courses/all.html


In [ ]:
def get_all_courses(driver):
    
    fout = open('a.json','w')
    out_list = []
    course_section = driver.find_element_by_xpath('//div[@aria-selected="true"]')
    course_list = course_section.find_elements_by_xpath('.//li[@class="courses-list__item"]')
    ncourse = 0
    for course in course_list:
        out = {}
        course_header = course.find_element_by_xpath(
            './/*[@class="courses-list__item-headers"]/a')
        out['title'] = course_header.get_attribute('title')
        out['url'] = course_header.get_attribute('href')
        course_meta = course.find_element_by_xpath(
            './/*[@class="courses-list__item-meta"]')
        out['author'] = course_meta.find_element_by_xpath(
            './p[@class="courses-list__item-authors"]/span/a').text
        out['level'] = course_meta.find_element_by_xpath(
            './p[@class="courses-list__item-level"]').text
        out['date'] = course_meta.find_element_by_xpath(
            './time[@class="courses-list__item-date"]').text
        out['level'] = course_meta.find_element_by_xpath(
            './time[@class="courses-list__item-duration"]').text
        print(json.dumps(out,sort_keys=True), file=fout)
        ncourse += 1
        if ncourse % 200 == 0:
            print('written {} lines'.format(ncourse))
            
    fout.close()
        

In [50]:
get_all_courses(driver)

{'title': 'Angular: The Big Picture', 'url': 'https://app.pluralsight.com/library/courses/ng-big-picture', 'level': '1h 6m', 'date': 'Dec 13, 2017', 'author': 'Joe Eames'}
{'title': 'Play by Play: Software Development in Swift - Beyond IOS', 'url': 'https://app.pluralsight.com/library/courses/play-by-play-software-development-swift-ios', 'level': '51m', 'date': 'Dec 13, 2017', 'author': 'Stephen Haunts'}
{'title': 'Design Patterns Playbook: Structural Patterns', 'url': 'https://app.pluralsight.com/library/courses/design-patterns-structural-playbook', 'level': '1h 5m', 'date': 'Dec 13, 2017', 'author': 'Paolo Perrotta'}
{'title': 'Extending AEM Advanced', 'url': 'https://app.pluralsight.com/library/courses/extending-aem-advanced', 'level': '6h 15m', 'date': 'Dec 13, 2017', 'author': 'Tyler Maynard'}
{'title': 'Microsoft Dynamics 365 Customer Engagement (CRM): Getting Started', 'url': 'https://app.pluralsight.com/library/courses/microsoft-dynamics-365-crm-getting-started', 'level': '2h 2

In [54]:
course_section = driver.find_element_by_xpath('//div[@aria-selected="true"]')

ncourse_expect = course_section.find_element_by_xpath(
        './/*[@class="l-search__results-page-info"]')
# .text.split()[1])

In [55]:
ncourse_expect.text

'Courses: 5451 results'

In [56]:
course_section.location

{'x': 140, 'y': 203}

In [64]:
driver.execute_script("window.scrollTo(0, 2081);")

In [60]:
elem = driver.find_element_by_xpath('//*[@id="ps-main"]/div/div/div/div[1]/div[2]/div[3]/div/div[2]/div/div[2]/ol/li[22]/div[1]/div[1]')

In [61]:
elem.location

{'x': 482, 'y': 2281}

In [65]:
elem.location['y']

2281

In [67]:
search.get_all_courses(driver, 'all.json')

TimeoutException: Message: timeout
  (Session info: chrome=63.0.3239.84)
  (Driver info: chromedriver=2.33.506106 (8a06c39c4582fbfbab6966dbb1c38a9173bfb1a2),platform=Mac OS X 10.13.1 x86_64)
